In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
in_class_labels = ['yes', 'no' , 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False,
                                               split_noise=True, n_fabricate_noise = 1600,
                                               split=0.85, novelty_det=True, novelty_det_in_class=in_class_labels)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
/usr/local/anaconda3/lib/python3.5/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.novelty_detection_spectrogram_multi_lstm_random_input_modify import *

In [4]:
graph_name = 'NoveltyDetectionSpectrogramMultiLSTMRandomInputModify'  # Specify graph class name

g_cnfg = Config('graph_01')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]

g_cnfg.random_modify_brightness_max_delta = 0.25
g_cnfg.random_modify_contrast_lower = 0.2
g_cnfg.random_modify_contrast_upper = 1.8

g_cnfg.lstm_state_sizes = [100, 200]
g_cnfg.n_hidden1 = 50
g_cnfg.n_hidden2 = 10

g_cnfg.sigmoid_accuracy_threshold = 0.95

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 0.7  # 1.0 means no decay

t_cnfg = Config('run_01')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.add_noise = (0.5, [s.spec_data for s in train.get_list_of_label('_background_noise_')])
t_cnfg.dropout_keep_prob = 0.75
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [5]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_01
X_img_h: 128
X_img_w: 71
lr_decay_rate: 0.7
lr_decay_steps: 1,000,000
lr_initial: 0.1
lstm_state_sizes: [100, 200]
n_hidden1: 50
n_hidden2: 10
random_modify_brightness_max_delta: 0.25
random_modify_contrast_lower: 0.2
random_modify_contrast_upper: 1.8
sigmoid_accuracy_threshold: 0.95

name: run_01
add_noise: (0.5, 'list of length 2,002')
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.75
early_stopping_patience: 0.1
le: None
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 379
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

NoveltyDetectionSpectrogramMultiLSTMRandomInputModify_graph_01_run_01
Training starts @ 12/23/2017 09:22:56
Epoch 1 Step 1 Best model saved @ 12/23/2017 09:23:03 [Ave valid] 0.706
Epoch 1 Step 1 ends @ 12/23/2017 09:23:03 [Train] 0.705, 64.0% [Valid] 65.0% [Ave valid] 0.706
Epoch 1 Step 50 Best model saved @ 12/23/2017 09:23:13 [Ave valid] 0.680
Epoch 1 Step 100 Best model saved @ 12/23/2017 09:23:

Epoch 12 Step 4,548 Best model saved @ 12/23/2017 09:39:17 [Ave valid] 0.246
Epoch 12 Step 4,548 ends @ 12/23/2017 09:39:17 [Train] 0.048, 85.7% [Valid] 78.5% [Ave valid] 0.246
Epoch 13 Step 4,550 Best model saved @ 12/23/2017 09:39:22 [Ave valid] 0.243
Epoch 13 Step 4,600 Best model saved @ 12/23/2017 09:39:32 [Ave valid] 0.240
Epoch 13 Step 4,650 Best model saved @ 12/23/2017 09:39:42 [Ave valid] 0.234
Epoch 13 Step 4,700 Best model saved @ 12/23/2017 09:39:53 [Ave valid] 0.230
Epoch 13 Step 4,750 Best model saved @ 12/23/2017 09:40:03 [Ave valid] 0.228
Epoch 13 Step 4,800 Best model saved @ 12/23/2017 09:40:13 [Ave valid] 0.226
Epoch 13 Step 4,850 Best model saved @ 12/23/2017 09:40:23 [Ave valid] 0.226
Epoch 13 Step 4,900 Best model saved @ 12/23/2017 09:40:34 [Ave valid] 0.218
Epoch 13 Step 4,927 ends @ 12/23/2017 09:40:41 [Train] 0.633, 57.1% [Valid] 68.7% [Ave valid] 0.258
Epoch 14 Step 5,300 Best model saved @ 12/23/2017 09:42:00 [Ave valid] 0.218
Epoch 14 Step 5,306 ends @ 12/

Epoch 44 Step 16,650 Best model saved @ 12/23/2017 10:21:59 [Ave valid] 0.099
Epoch 44 Step 16,676 Best model saved @ 12/23/2017 10:22:06 [Ave valid] 0.098
Epoch 44 Step 16,676 ends @ 12/23/2017 10:22:06 [Train] 0.028, 100.0% [Valid] 90.3% [Ave valid] 0.098
Epoch 45 Step 16,700 Best model saved @ 12/23/2017 10:22:13 [Ave valid] 0.098
Epoch 45 Step 16,750 Best model saved @ 12/23/2017 10:22:23 [Ave valid] 0.097
Epoch 45 Step 16,800 Best model saved @ 12/23/2017 10:22:33 [Ave valid] 0.096
Epoch 45 Step 16,850 Best model saved @ 12/23/2017 10:22:43 [Ave valid] 0.094
INFO:tensorflow:checkpoints/NoveltyDetectionSpectrogramMultiLSTMRandomInputModify_graph_01_run_01/hourly/model-16950 is not in all_model_checkpoint_paths. Manually adding it.
Epoch 45 Step 16,950 Hourly model saved @ 12/23/2017 10:23:03
Epoch 45 Step 17,055 ends @ 12/23/2017 10:23:26 [Train] 0.006, 100.0% [Valid] 94.3% [Ave valid] 0.095
Epoch 46 Step 17,434 Best model saved @ 12/23/2017 10:24:42 [Ave valid] 0.094
Epoch 46 Step

Epoch 83 Step 31,100 Best model saved @ 12/23/2017 11:12:25 [Ave valid] 0.070
Epoch 83 Step 31,150 Best model saved @ 12/23/2017 11:12:35 [Ave valid] 0.069
Epoch 83 Step 31,457 ends @ 12/23/2017 11:13:40 [Train] 0.011, 92.9% [Valid] 96.0% [Ave valid] 0.072
Epoch 84 Step 31,836 ends @ 12/23/2017 11:14:59 [Train] 0.116, 85.7% [Valid] 95.0% [Ave valid] 0.084
Epoch 85 Step 32,215 ends @ 12/23/2017 11:16:20 [Train] 0.067, 71.4% [Valid] 91.0% [Ave valid] 0.076
Epoch 86 Step 32,594 ends @ 12/23/2017 11:17:39 [Train] 0.060, 100.0% [Valid] 96.3% [Ave valid] 0.090
Epoch 87 Step 32,973 ends @ 12/23/2017 11:19:00 [Train] 0.001, 100.0% [Valid] 96.0% [Ave valid] 0.072
Epoch 88 Step 33,352 ends @ 12/23/2017 11:20:24 [Train] 0.001, 100.0% [Valid] 93.9% [Ave valid] 0.075
Epoch 89 Step 33,731 ends @ 12/23/2017 11:21:42 [Train] 0.016, 85.7% [Valid] 96.5% [Ave valid] 0.071
INFO:tensorflow:checkpoints/NoveltyDetectionSpectrogramMultiLSTMRandomInputModify_graph_01_run_01/hourly/model-34110 is not in all_mod

Epoch 137 Step 51,650 Hourly model saved @ 12/23/2017 12:23:14
Epoch 137 Step 51,923 Best model saved @ 12/23/2017 12:24:07 [Ave valid] 0.060
Epoch 137 Step 51,923 ends @ 12/23/2017 12:24:07 [Train] 0.024, 92.9% [Valid] 97.0% [Ave valid] 0.060
Epoch 138 Step 51,950 Best model saved @ 12/23/2017 12:24:14 [Ave valid] 0.060
Epoch 138 Step 52,302 ends @ 12/23/2017 12:25:24 [Train] 0.001, 100.0% [Valid] 95.6% [Ave valid] 0.063
Epoch 139 Step 52,681 ends @ 12/23/2017 12:26:37 [Train] 0.000, 100.0% [Valid] 97.1% [Ave valid] 0.064
Epoch 140 Step 53,060 ends @ 12/23/2017 12:27:53 [Train] 0.001, 100.0% [Valid] 97.0% [Ave valid] 0.063
Epoch 141 Step 53,439 ends @ 12/23/2017 12:29:07 [Train] 0.008, 100.0% [Valid] 97.2% [Ave valid] 0.061
Epoch 142 Step 53,818 ends @ 12/23/2017 12:30:23 [Train] 0.000, 100.0% [Valid] 96.8% [Ave valid] 0.060
Epoch 143 Step 54,197 ends @ 12/23/2017 12:31:36 [Train] 0.006, 92.9% [Valid] 97.5% [Ave valid] 0.067
Epoch 144 Step 54,576 ends @ 12/23/2017 12:32:53 [Train] 0.0

Epoch 207 Step 78,100 Best model saved @ 12/23/2017 13:50:36 [Ave valid] 0.058
Epoch 207 Step 78,150 Best model saved @ 12/23/2017 13:50:46 [Ave valid] 0.057
Epoch 207 Step 78,200 Best model saved @ 12/23/2017 13:50:55 [Ave valid] 0.057
Epoch 207 Step 78,250 Best model saved @ 12/23/2017 13:51:05 [Ave valid] 0.056
Epoch 207 Step 78,300 Best model saved @ 12/23/2017 13:51:14 [Ave valid] 0.056
Epoch 207 Step 78,453 ends @ 12/23/2017 13:51:46 [Train] 0.286, 85.7% [Valid] 92.0% [Ave valid] 0.073
Epoch 208 Step 78,832 ends @ 12/23/2017 13:53:00 [Train] 0.003, 100.0% [Valid] 97.7% [Ave valid] 0.060
Epoch 209 Step 79,211 ends @ 12/23/2017 13:54:16 [Train] 0.000, 100.0% [Valid] 97.2% [Ave valid] 0.057
Epoch 210 Step 79,590 ends @ 12/23/2017 13:55:30 [Train] 0.130, 85.7% [Valid] 96.3% [Ave valid] 0.064
Epoch 211 Step 79,969 ends @ 12/23/2017 13:56:46 [Train] 0.001, 100.0% [Valid] 96.2% [Ave valid] 0.058
Epoch 212 Step 80,348 ends @ 12/23/2017 13:58:00 [Train] 0.000, 100.0% [Valid] 97.4% [Ave va

KeyboardInterrupt: 